In [3]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.8 MB/s eta 0:00:00


In [4]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [5]:
print(tf.__version__)

2.15.0


In [6]:
fashion_mnist=keras.datasets.fashion_mnist

In [7]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [8]:
train_images=train_images/255.0
test_images=test_images/255.0

In [9]:
train_images[0].shape

(28, 28)

In [10]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [11]:
train_images

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

In [12]:
def build_model(hp):
  model=keras.Sequential([
      keras.layers.Conv2D(
          filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
          kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
          activation='relu',
          input_shape=(28,28,1)

      ),
      keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
          kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
          activation='relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
      units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
      ),
      keras.layers.Dense(10,activation='softmax')
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  return model

In [13]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-13-dce6bfdf082c>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [14]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,
                          directory='output',project_name="Mnist Fashion")

In [15]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.8603333234786987

Best val_accuracy So Far: 0.9086666703224182
Total elapsed time: 00h 03m 08s


In [16]:
model=tuner_search.get_best_models(num_models=1)[0]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 128)       1280      
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        36896     
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 96)                1769568   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 1808714 (6.90 MB)
Trainable params: 1808714 (6.90 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 10s 5ms/step - loss: 0.1691 - accuracy: 0.9380 - val_loss: 0.2712 - val_accuracy: 0.9112
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1380 - accuracy: 0.9496 - val_loss: 0.2763 - val_accuracy: 0.9142
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1155 - accuracy: 0.9571 - val_loss: 0.2870 - val_accuracy: 0.9077
Epoch 7/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0942 - accuracy: 0.9652 - val_loss: 0.3154 - val_accuracy: 0.9065
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0812 - accuracy: 0.9702 - val_loss: 0.3653 - val_accuracy: 0.9072
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0681 - accuracy: 0.9760 - val_loss: 0.3954 - val_accuracy: 0.9045
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0586 - accuracy: 0.9789 - val_loss: 0.4550 - val_accura